In [2]:
import pandas as pd
import numpy as np

In [57]:
sessions_df = pd.read_json('/home/janek/Projekty/IUM/data/raw/sessions.jsonl', lines=True)
sessions_df


,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,124,2021-01-01 18:24:14,102,1002,VIEW_PRODUCT,0,NaN
1,124,2021-01-01 18:27:37,102,1003,VIEW_PRODUCT,0,NaN
2,124,2021-01-01 18:28:11,102,1075,VIEW_PRODUCT,0,NaN
3,124,2021-01-01 18:29:03,102,1076,VIEW_PRODUCT,0,NaN
4,124,2021-01-01 18:30:46,102,1077,VIEW_PRODUCT,0,NaN
...,...,...,...,...,...,...,...
59882,9793,2021-12-03 00:39:12,301,1035,VIEW_PRODUCT,20,NaN
59883,9793,2021-12-03 00:43:38,301,1067,VIEW_PRODUCT,20,NaN
59884,9793,2021-12-03 00:47:05,301,1067,BUY_PRODUCT,20,24236.0
59885,9794,2021-12-06 19:59:53,301,1318,VIEW_PRODUCT,15,NaN


In [45]:
sessions_without_buy_event = sessions_df[sessions_df['event_type'] == 'VIEW_PRODUCT']

In [72]:
grouped_sessions = sessions_without_buy_event.groupby(['session_id', 'user_id']).count()
grouped_sessions.reset_index(level="user_id")
del grouped_sessions['timestamp']
del grouped_sessions['event_type']
del grouped_sessions['offered_discount']
del grouped_sessions['purchase_id']
grouped_sessions = grouped_sessions.rename(columns={'product_id': "total_views"}).reset_index(level=["user_id"])
grouped_sessions

,user_id,total_views
session_id,,
124,102,9
125,102,2
126,102,10
127,102,10
128,102,2
...,...,...
9790,301,1
9791,301,9
9792,301,2


In [73]:
grouped_sessions['unique_products'] = sessions_df.groupby('session_id').product_id.agg('unique')


In [74]:
grouped_sessions

,user_id,total_views,unique_products
session_id,,,
124,102,9,"[1002, 1003, 1075, 1076, 1077, 1078, 1079, 108..."
125,102,2,[1283]
126,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105..."
127,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105..."
128,102,2,"[1278, 1281]"
...,...,...,...
9790,301,1,[1283]
9791,301,9,"[1040, 1041, 1042, 1043, 1044, 1045, 1046, 104..."
9792,301,2,"[1276, 1277]"


In [75]:
time_of_each_session = sessions_df.groupby(['session_id']).timestamp.agg([min, max])
time_of_each_session = (time_of_each_session['max'] - time_of_each_session['min']).dt.total_seconds().div(60).astype(int)
grouped_sessions['time_in_minutes'] = time_of_each_session
grouped_sessions

,user_id,total_views,unique_products,time_in_minutes
session_id,,,,
124,102,9,"[1002, 1003, 1075, 1076, 1077, 1078, 1079, 108...",19
125,102,2,[1283],4
126,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105...",15
127,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105...",19
128,102,2,"[1278, 1281]",1
...,...,...,...,...
9790,301,1,[1283],0
9791,301,9,"[1040, 1041, 1042, 1043, 1044, 1045, 1046, 104...",22
9792,301,2,"[1276, 1277]",6


In [98]:
sessions_when_bought_grouped = sessions_df[sessions_df.event_type == 'BUY_PRODUCT'].groupby('session_id').count()['event_type']
data_to_model = grouped_sessions.join(sessions_when_bought_grouped, rsuffix="bought").fillna(0).rename(columns={"event_type": "ended_with_bought"})


In [99]:
import json
f = open('/home/janek/Projekty/IUM/data/processed/session_ratio_per_user.json')
ratio = json.load(f)
ratio

{'102': 0.3972602739726027,
 '103': 0.3333333333333333,
 '104': 0.0,
 '105': 0.5,
 '106': 0.5121951219512195,
 '107': 0.5357142857142857,
 '108': 0.45901639344262296,
 '109': 0.5543478260869565,
 '110': 0.4533333333333333,
 '111': 0.49382716049382713,
 '112': 0.5,
 '113': 0.5263157894736842,
 '114': 0.0,
 '115': 0.3442622950819672,
 '116': 0.3137254901960784,
 '117': 0.3888888888888889,
 '118': 0.4155844155844156,
 '119': 0.48936170212765956,
 '120': 0.4745762711864407,
 '121': 0.36065573770491804,
 '122': 0.4406779661016949,
 '123': 0.42857142857142855,
 '124': 0.3333333333333333,
 '125': 0.4642857142857143,
 '126': 0.4225352112676056,
 '127': 0.4117647058823529,
 '128': 0.35555555555555557,
 '129': 0.30303030303030304,
 '130': 0.3181818181818182,
 '131': 0.45652173913043476,
 '132': 0.42857142857142855,
 '133': 0.3333333333333333,
 '134': 0.5217391304347826,
 '135': 0.2727272727272727,
 '136': 0.44594594594594594,
 '137': 0.3684210526315789,
 '138': 0.5070422535211268,
 '139': 0.4945

In [100]:
data_to_model['user_bought_to_total_sessions_ratio'] = data_to_model['user_id'].map(lambda x: ratio[str(x)])

In [101]:
data_to_model.to_csv('')

,user_id,total_views,unique_products,time_in_minutes,ended_with_bought,user_bought_to_total_sessions_ratio
session_id,,,,,,
124,102,9,"[1002, 1003, 1075, 1076, 1077, 1078, 1079, 108...",19,1.0,0.397260
125,102,2,[1283],4,0.0,0.397260
126,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105...",15,0.0,0.397260
127,102,10,"[1048, 1049, 1050, 1051, 1052, 1053, 1054, 105...",19,0.0,0.397260
128,102,2,"[1278, 1281]",1,1.0,0.397260
...,...,...,...,...,...,...
9790,301,1,[1283],0,0.0,0.348837
9791,301,9,"[1040, 1041, 1042, 1043, 1044, 1045, 1046, 104...",22,1.0,0.348837
9792,301,2,"[1276, 1277]",6,1.0,0.348837


In [6]:
sessions_df = sessions_df.join(time_of_each_session, on='session_id')
sessions_df

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,min,max
0,124,2021-01-01 18:24:14,102,1002,VIEW_PRODUCT,0,NaN,2021-01-01 18:24:14,2021-01-01 18:43:28
1,124,2021-01-01 18:27:37,102,1003,VIEW_PRODUCT,0,NaN,2021-01-01 18:24:14,2021-01-01 18:43:28
2,124,2021-01-01 18:28:11,102,1075,VIEW_PRODUCT,0,NaN,2021-01-01 18:24:14,2021-01-01 18:43:28
3,124,2021-01-01 18:29:03,102,1076,VIEW_PRODUCT,0,NaN,2021-01-01 18:24:14,2021-01-01 18:43:28
4,124,2021-01-01 18:30:46,102,1077,VIEW_PRODUCT,0,NaN,2021-01-01 18:24:14,2021-01-01 18:43:28
...,...,...,...,...,...,...,...,...,...
59882,9793,2021-12-03 00:39:12,301,1035,VIEW_PRODUCT,20,NaN,2021-12-03 00:39:12,2021-12-03 00:47:05
59883,9793,2021-12-03 00:43:38,301,1067,VIEW_PRODUCT,20,NaN,2021-12-03 00:39:12,2021-12-03 00:47:05
59884,9793,2021-12-03 00:47:05,301,1067,BUY_PRODUCT,20,24236.0,2021-12-03 00:39:12,2021-12-03 00:47:05
59885,9794,2021-12-06 19:59:53,301,1318,VIEW_PRODUCT,15,NaN,2021-12-06 19:59:53,2021-12-06 20:04:07


In [7]:
def calculate_minutes_since_session_begin(row):
    difference = pd.Timestamp(row['timestamp']) - pd.Timestamp(row['min'])
    row['minutes_since_session_begin'] = difference.seconds // 60 % 60
    return row

result_df = sessions_df.apply(calculate_minutes_since_session_begin, axis='columns')

In [8]:
del result_df['timestamp']
del result_df['user_id']
del result_df['purchase_id']
del result_df['min']
del result_df['max']

In [9]:
result_df

,session_id,product_id,event_type,offered_discount,minutes_since_session_begin
0,124,1002,VIEW_PRODUCT,0,0
1,124,1003,VIEW_PRODUCT,0,3
2,124,1075,VIEW_PRODUCT,0,3
3,124,1076,VIEW_PRODUCT,0,4
4,124,1077,VIEW_PRODUCT,0,6
...,...,...,...,...,...
59882,9793,1035,VIEW_PRODUCT,20,0
59883,9793,1067,VIEW_PRODUCT,20,4
59884,9793,1067,BUY_PRODUCT,20,7
59885,9794,1318,VIEW_PRODUCT,15,0


In [14]:
result_df.event_type = result_df.event_type.map(lambda event: 0 if event == 'VIEW_PRODUCT' else 1)

In [28]:
result_df

,session_id,product_id,event_type,offered_discount,minutes_since_session_begin
0,124,1002,0,0,0
1,124,1003,0,0,3
2,124,1075,0,0,3
3,124,1076,0,0,4
4,124,1077,0,0,6
...,...,...,...,...,...
59882,9793,1035,0,20,0
59883,9793,1067,0,20,4
59884,9793,1067,1,20,7
59885,9794,1318,0,15,0


In [16]:
products_df = pd.read_csv('/home/janek/Projekty/IUM/data/processed/products_with_one_hot_encoding.csv')

In [18]:
del products_df['Unnamed: 0']

In [22]:
products_df

,product_id,price,category_path_Akcesoria telefoniczne,category_path_Anteny RTV,category_path_Audio,category_path_Biurowe urządzenia wielofunkcyjne,category_path_Drukarki i skanery,category_path_Gry PlayStation3,category_path_Gry Xbox 360,category_path_Gry i konsole,...,category_path_Słuchawki,category_path_Tablety,category_path_Tablety i akcesoria,category_path_Telefony i akcesoria,category_path_Telefony komórkowe,category_path_Telefony stacjonarne,category_path_Telewizory i akcesoria,category_path_Video,category_path_Zestawy głośnomówiące,category_path_Zestawy słuchawkowe
0,1001,117.94,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1002,6145.50,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1003,22917.00,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1004,149.97,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1005,149.97,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,1315,164.97,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
315,1316,747.00,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
316,1317,3911.88,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
317,1318,1659.00,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [23]:
products_df.set_index('product_id')

,price,category_path_Akcesoria telefoniczne,category_path_Anteny RTV,category_path_Audio,category_path_Biurowe urządzenia wielofunkcyjne,category_path_Drukarki i skanery,category_path_Gry PlayStation3,category_path_Gry Xbox 360,category_path_Gry i konsole,category_path_Gry komputerowe,...,category_path_Słuchawki,category_path_Tablety,category_path_Tablety i akcesoria,category_path_Telefony i akcesoria,category_path_Telefony komórkowe,category_path_Telefony stacjonarne,category_path_Telewizory i akcesoria,category_path_Video,category_path_Zestawy głośnomówiące,category_path_Zestawy słuchawkowe
product_id,,,,,,,,,,,,,,,,,,,,,
1001,117.94,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1002,6145.50,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,22917.00,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,149.97,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1005,149.97,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,164.97,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1316,747.00,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1317,3911.88,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [32]:
result_df.merge(products_df).sort_values(by='session_id').to_csv('/home/janek/Projekty/IUM/data/processed/result_dataset.csv')